In [5]:
import os
import re
import pandas as pd
import numpy as np
import tqdm
import openai

In [3]:
openai.api_type = "azure"
openai.api_base = ""
openai.api_version = "2023-03-15-preview"
openai.api_key = ""


In [ ]:
stage = 'stage2'

In [ ]:
ipfile = f'{stage}_train_processed_data.json'
opfile = f'{stage}_with_adversarial_TRAIN.csv'

opfolder = 'adversarial_simple'   # keep as is.

In [ ]:
os.system(f'mkdir {opfolder}')

### simple prompt

In [ ]:
def get_simple_prompt(prompt,prefix=None):

    if(prefix is not None):
        default = prefix
    else:
        default = '''Augment the passage as shown by adding one new variable. 
Rules:
1. The new variable must change values over the course of the augmented passage.
2. Old and new value of the new variable should be explicitly mentioned.
3. The augmented passage should add numerical information regarding variables not 4. measured or derived from variables in the passage. 
4. The new variable must be one of the following types: [Volume, Humidity, Temperature, Weight, Luminosity, Density, Speed, Area]. 
5. The new variable type must not be present in the passage before augmentation.
6. The new variable must not share the same physical unit as any of the original variables. 
7. The new variable must not be related to or derived from the existing variables in the passage.
8. The augmented text should not add any numerical information about existing variables that is not already mentioned explicitly in the original passage.
9. Add no more than one sentence.

Example 1:
Passage:  My car gets 20 miles per gallon.
Existing Variables: Fuel efficiency
Augmented: My car, which gets 20 miles for each gallon, when I drive at 120 miles per hour. I start going at 10 feet a minute. 
New Variables: Speed (changes from 120 to 10)

Explanation: the new variable speed is added. It is independent of the Fuel efficiency variable in the original passage. Speed cannot be derived from the original passage. The units for speed are not related to the units for fuel efficiency. The augmented passage does not add any information about existing variables (fuel efficiency) at all, fulfilling Rule 8. 

Example 2:
Passage:  There are 64 pigs in the barn. Some more came in, now there are 86 pigs.
Existing Variables: Number of pigs (changes from 64 to 86)
Augmented: In the barn, where the temperature is a cozy 72 degrees Fahrenheit, there are 64 pigs. Some more came in. The temperature goes up to 83 degrees fahrenheit for 86 pigs. 
New variables: Temperature (changes from 72 to 83)

Explanation: the new variable temperature is added that is independent of the number of pigs variable. The units for temperature are not related to the units for the number of pigs.  The augmented passage does not add any information about existing variables (number of pigs) at all, fulfilling Rule 8. 

Now output the existing variables and augment the passage below following all rules. Also print the new variables in the augmented passage.
Passage: {}
Existing Variables:
'''.format(prompt)


    return default


In [ ]:
def postprocess_simple(inp: str):
    return inp.split('Augmented:')[1].split('New variables:')[0]

### complex prompt

In [4]:
def get_complex_prompt(prompt,prefix=None):

    if(prefix is not None):
        default = prefix
    else:
        default = '''Augment the passage as shown by adding one new variable. The new variable must change values, and the new variables must be in a different unit to the original passage. 


Passage:  My car gets 20 miles per gallon.
Variables: Fuel efficiency
Augmented: My car, which gets 20 miles for each gallon, when I drive at 120 miles per hour. I start going at 10 feet a minute. 
New Variables: Speed (changes from 120 to 10)


Passage:  There are 64 pigs in the barn. Some more came in, now there are 86 pigs.
Variables: Number of pigs (changes from 64 to 86)
Augmented: In the barn, where the temperature is a cozy 72 degrees Fahrenheit, there are 64 pigs. Some more came in. The temperature goes up to 83 degrees fahrenheit for 86 pigs. 
New variables: Temperature (changes from 72 to 83)


In example 1, the new variable speed is added. It is independent of the Fuel efficiency variable in the original passage. Speed cannot be derived from the original passage. In example 2, the new variable temperature is added that is independent of the number of pigs variable.

The augmented passage should add numerical information regarding variables not measured or derived from variables in the passage. The new variables must not share the same physical unit as the original variable. 


Now output the variables and augment the passage below following this rule. Do not add any information about the variables in passage, which is not there already. the new variables must have clear before and after change values.


Passage: {}
Variables:'''.format(prompt)


    return default


In [5]:

def get_inference(prompt_text, returnPrompt=False):

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages = [

                        {"role": "system", "content": "You are a mathematics and linguistics professor setting a test."},

                        {"role": "user", "content": prompt_text}

                    ],
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)

    ans = response["choices"][0]["message"]["content"].split("\n")

    if(returnPrompt):
        return [prompt_text,ans]
    else:
        return [ans] 

In [6]:
file = pd.read_json(ipfile)
file.info()
passage_ids = file.columns

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, passage to qa_pairs
Columns: 984 entries, passage_0 to passage_983
dtypes: object(984)
memory usage: 15.4+ KB


In [9]:
ip_list2 = passage_ids

count = 0

batch_size = 50  

In [25]:

while len(ip_list2) > 0:
    temp_list = ip_list2[:batch_size]

    para=[]
    question=[]
    answer=[]
    query_id=[]
    is_adversarial=[]
    passage_ids_L=[]

    print('creating shard ',count)
 

    for p in tqdm.tqdm(temp_list):
        passage_ids_L.append(p)
        passage_ids_L.append(p)

        para.append(file[p]['passage'])
        question.append(file[p]['qa_pairs'][0]['question'])
        answer.append(file[p]['qa_pairs'][0]['answer'])
        query_id.append(file[p]['qa_pairs'][0]['query_id'])
        is_adversarial.append(False)

        para.append(postprocess_simple(get_inference(get_simple_prompt(file[p]['passage']))))
        question.append(file[p]['qa_pairs'][0]['question'])
        answer.append(file[p]['qa_pairs'][0]['answer'])
        query_id.append(file[p]['qa_pairs'][0]['query_id'])
        is_adversarial.append(True)
    
    dff = pd.DataFrame(data={'passage_id':passage_ids_L,
                        'passage':para,
                        'question':question,
                        'answer':answer,
                        'query_id':query_id,
                        'isAdversarial':is_adversarial})

    dff.to_csv(os.path.join(opfolder,opfile.replace('.csv','SHARD_{}.csv'.format(count))),index=False)
    
    count+=1
    print("shard number:",count,"Done")
    ip_list2 = ip_list2[batch_size:]

creating shard  4


 26%|██▌       | 13/50 [00:37<01:47,  2.92s/it]